In [14]:
'''Felix Andersson, Janine de Vries, DV2626'''

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer


In [15]:
# Steg 1: Ladda dataset
exercise_data = pd.read_csv('gym.csv', delimiter=';')
important_data = exercise_data[['Exercise Name', 'Main_muscle', 'Mechanics', 'Target_Muscles', 'Synergist_Muscles', 'Stabilizer_Muscles', 'Antagonist_Muscles', 'Dynamic_Stabilizer_Muscles', 'Difficulty (1-5)', 'Secondary Muscles']].copy()
important_data.head()

,Exercise Name,Main_muscle,Mechanics,Target_Muscles,Synergist_Muscles,Stabilizer_Muscles,Antagonist_Muscles,Dynamic_Stabilizer_Muscles,Difficulty (1-5),Secondary Muscles
0,Front Raise,Shoulder,Isolated,"Anterior Deltoid,","Pectoralis Major, Clavicular, Deltoid, Lateral...","Trapezius, Upper, Levator Scapulae, Wrist Exte...",NaN,NaN,2,"Anterior Deltoid, Serratus Anterior"
1,Military Press,Shoulder,Compound,"Anterior Deltoid,","Pectoralis Major, Clavicular, Triceps Brachii,...","Trapezius, Upper, Levator Scapulae,",NaN,"Triceps, Long Head, Biceps Brachii, Short Head,",4,"Lateral Deltoid, Triceps Brachii"
2,Military Press: Seated,Shoulder,Compound,"Anterior Deltoid,","Deltoid, Lateral, Supraspinatus, Triceps Brach...","Trapezius, Upper, Levator Scapulae,",NaN,"Triceps, Long Head,",3,"Lateral Deltoid, Triceps Brachii"
3,Front Raise,Shoulder,Isolated,"Anterior Deltoid,","Pectoralis Major, Clavicular, Deltoid, Lateral...","Trapezius, Upper, Levator Scapulae, Wrist Exte...",NaN,NaN,2,"Anterior Deltoid, Serratus Anterior"
4,Front Raise: Alternating,Shoulder,Isolated,"Anterior Deltoid,","Pectoralis Major, Clavicular, Deltoid, Lateral...","Trapezius, Upper, Levator Scapulae, Wrist Exte...",NaN,NaN,2,"Anterior Deltoid, Serratus Anterior"


In [16]:
columns_to_encode = [
    'Main_muscle', 'Mechanics','Target_Muscles', 'Synergist_Muscles', 'Stabilizer_Muscles', 'Antagonist_Muscles', 'Dynamic_Stabilizer_Muscles', 'Secondary Muscles'
]

# Loop through each column (excluding Difficulty) and apply transformations
for column in columns_to_encode:
    # Fill NaN values with empty strings
    important_data.loc[:, column] = important_data[column].fillna("")

    # Split multi-value strings into lists
    important_data.loc[:, column] = important_data[column].apply(lambda x: x.split(", ") if x else [])

    # Apply MultiLabelBinarizer to encode the muscles into binary values
    mlb = MultiLabelBinarizer()
    binary_matrix = mlb.fit_transform(important_data[column])

    # Replace the column with the binary list
    important_data.loc[:, f"ID_{column}"] = binary_matrix.tolist()

    # Drop the original column (if needed)
    important_data = important_data.drop(columns=[column])

important_data.to_csv('gym_ID.csv', index=False)
important_data.head()

,Exercise Name,Difficulty (1-5),ID_Main_muscle,ID_Mechanics,ID_Target_Muscles,ID_Synergist_Muscles,ID_Stabilizer_Muscles,ID_Antagonist_Muscles,ID_Dynamic_Stabilizer_Muscles,ID_Secondary Muscles
0,Front Raise,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]","[0, 1]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,Military Press,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]","[1, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,Military Press: Seated,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]","[1, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3,Front Raise,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]","[0, 1]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Front Raise: Alternating,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]","[0, 1]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import ast

# Step 1: Load and Prepare the Dataset
data = pd.read_csv("training_knn.csv")

# Convert array-like strings to actual lists
for col in ["ID_Main_muscle", "ID_Mechanics", "ID_Target_Muscles", 
            "ID_Synergist_Muscles", "ID_Stabilizer_Muscles", 
            "ID_Antagonist_Muscles", "ID_Dynamic_Stabilizer_Muscles", 
            "ID_Secondary Muscles"]:
    data[col] = data[col].apply(ast.literal_eval).apply(np.array)

# Step 2: Feature Engineering
# Flatten array columns into separate features
array_columns = ["ID_Main_muscle", "ID_Mechanics", "ID_Target_Muscles", 
                 "ID_Synergist_Muscles", "ID_Stabilizer_Muscles", 
                 "ID_Antagonist_Muscles", "ID_Dynamic_Stabilizer_Muscles", 
                 "ID_Secondary Muscles"]
flattened_features = []
for col in array_columns:
    arrays = np.stack(data[col].values)  # Stack arrays for easier handling
    array_df = pd.DataFrame(arrays, columns=[f"{col}_{i}" for i in range(arrays.shape[1])])
    flattened_features.append(array_df)

# Concatenate flattened features with other columns
data_flattened = pd.concat([data.drop(array_columns, axis=1)] + flattened_features, axis=1)

# Step 3: Define Features and Target
X = data_flattened.drop(["Exercise Name","Difficulty (1-5)","score"], axis=1)
y = data_flattened["score"]

# Step 4: Split the Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)
exercise_names = data_flattened["Exercise Name"].iloc[X_test.index].reset_index(drop=True)

# Step 5: Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 6: Train KNN Model
knn = KNeighborsRegressor(n_neighbors=3)
knn.fit(X_train_scaled, y_train)
y_pred = knn.predict(X_test_scaled)

# Calculate RMSE
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"Root Mean Squared Error (RMSE): {rmse}")

# Step 7: Adjust Scores
def adjust_score(predicted_scores, user_fitness_level, exercise_difficulty, max_difference=4):
    """
    Adjust predicted scores based on user's fitness level and exercise difficulty.
    """
    adjusted_scores = []
    for pred, difficulty in zip(predicted_scores, exercise_difficulty):
        diff_ratio = abs(user_fitness_level - difficulty) / max_difference
        factor = 1 - (diff_ratio ** 2)  # Smooth penalty
        factor = max(0.5, factor)  # Ensure minimum adjustment factor is 0.5
        adjusted_scores.append(pred * factor)
    return adjusted_scores

# Get User Input
while True:
    try:
        user_fitness_level = int(input("Enter your fitness level (1=Beginner to 5=Advanced): "))
        if 1 <= user_fitness_level <= 5:
            break
        else:
            print("Please enter a valid fitness level (1 to 5).")
    except ValueError:
        print("Invalid input. Please enter a number between 1 and 5.")

# Apply adjustments
exercise_difficulty = data_flattened["Difficulty (1-5)"].iloc[X_test.index].reset_index(drop=True)
adjusted_scores = adjust_score(y_pred, user_fitness_level, exercise_difficulty)

# Add predictions for review
X_test["Predicted Score"] = adjusted_scores

# Print each prediction with exercise name and actual score
print("\nPredictions with Adjusted Scores:")
for index, row in X_test.iterrows():
    exercise_name = exercise_names.iloc[index]
    print(f"Exercise: {exercise_name}, Predicted Score: {row['Predicted Score']:.2f}, Actual Score: {y_test.iloc[index]}")


C:\Users\felea\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Root Mean Squared Error (RMSE): 2.2694331445067117

Predictions with Adjusted Scores:
Exercise: Bench Press, Predicted Score: 8.00, Actual Score: 8
Exercise: Incline Bench Press, Predicted Score: 5.67, Actual Score: 9
Exercise: Chest Dip, Predicted Score: 6.00, Actual Score: 8
Exercise: Bench Press, Predicted Score: 5.67, Actual Score: 5
Exercise: Incline Bench Press, Predicted Score: 6.67, Actual Score: 7
Exercise: Decline Bench Press, Predicted Score: 4.33, Actual Score: 8
Exercise: Presses:  Decline Chest Press, Predicted Score: 5.33, Actual Score: 2
Exercise: Push-up, Predicted Score: 5.00, Actual Score: 3
Exercise: Fly, Predicted Score: 6.56, Actual Score: 9
Exercise: Flies:  Lying Fly, Predicted Score: 5.62, Actual Score: 8
Exercise: Flies:  Pec Deck Fly, Predicted Score: 6.25, Actual Score: 5
Exercise: Military Press, Predicted Score: 7.81, Actual Score: 8
Exercise: Military Press, Predicted Score: 3.44, Actual Score: 8
Exercise: Military Press:  Seated, Predicted Score: 6.00, A

In [39]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import ast

data = pd.read_csv("training_knn.csv")

array_columns = ["ID_Main_muscle", "ID_Mechanics", "ID_Target_Muscles", 
                 "ID_Synergist_Muscles", "ID_Stabilizer_Muscles", 
                 "ID_Antagonist_Muscles", "ID_Dynamic_Stabilizer_Muscles", 
                 "ID_Secondary Muscles"]
for col in array_columns:
    data[col] = data[col].apply(ast.literal_eval).apply(np.array)

flattened_features = []
for col in array_columns:
    arrays = np.stack(data[col].values)  
    array_df = pd.DataFrame(arrays, columns=[f"{col}_{i}" for i in range(arrays.shape[1])])
    flattened_features.append(array_df)

data_flattened = pd.concat([data.drop(array_columns, axis=1)] + flattened_features, axis=1)

X = data_flattened.drop(["Exercise Name", "Difficulty (1-5)", "score"], axis=1)
y = data_flattened["score"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
knn = KNeighborsRegressor(n_neighbors=3)
knn.fit(X_scaled, y)

data_test = pd.read_csv("gym_ID.csv")

for col in array_columns:
    data_test[col] = data_test[col].apply(ast.literal_eval).apply(np.array)


flattened_features_test = []
for col in array_columns:
    arrays_test = np.stack(data_test[col].values)
    array_df_test = pd.DataFrame(arrays_test, columns=[f"{col}_{i}" for i in range(arrays_test.shape[1])])
    flattened_features_test.append(array_df_test)

data_test_flattened = pd.concat([data_test.drop(array_columns, axis=1)] + flattened_features_test, axis=1)

X_test = data_test_flattened.drop(["Exercise Name", "Difficulty (1-5)"], axis=1)
X_test_scaled = scaler.transform(X_test)

y_pred_test = knn.predict(X_test_scaled)

def adjust_score(predicted_scores, user_fitness_level, exercise_difficulty, max_difference=4):
    adjusted_scores = []
    for pred, difficulty in zip(predicted_scores, exercise_difficulty):
        diff_ratio = abs(user_fitness_level - difficulty) / max_difference
        factor = 1 - (diff_ratio ** 2) 
        factor = max(0.5, factor) 
        adjusted_scores.append(pred * factor)
    return adjusted_scores

user_fitness_level = 5 

exercise_difficulty = data_test_flattened["Difficulty (1-5)"]
adjusted_scores_test = adjust_score(y_pred_test, user_fitness_level, exercise_difficulty)

data_test_flattened["Predicted Score"] = adjusted_scores_test

sorted_results = data_test_flattened.sort_values(by="Predicted Score", ascending=False)
print("recomendede exercise: ", sorted_results["Exercise Name"].iloc[0])
sorted_results.head()

recomendede exercise:  Deadlift


,Exercise Name,Difficulty (1-5),ID_Main_muscle_0,ID_Main_muscle_1,ID_Main_muscle_2,ID_Main_muscle_3,ID_Main_muscle_4,ID_Main_muscle_5,ID_Main_muscle_6,ID_Main_muscle_7,...,ID_Secondary Muscles_26,ID_Secondary Muscles_27,ID_Secondary Muscles_28,ID_Secondary Muscles_29,ID_Secondary Muscles_30,ID_Secondary Muscles_31,ID_Secondary Muscles_32,ID_Secondary Muscles_33,ID_Secondary Muscles_34,Predicted Score
346,Deadlift,5,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,8.333333
380,Deadlift,5,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,8.333333
326,Squat: Full Squat,5,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,8.333333
316,Deadlifts,5,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,8.333333
303,Decline Push-up: on stability ball,4,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,8.125000
